In [2]:
import argparse
import os
import logging
import tensorflow as tf
import numpy as np

In [3]:
#Loading the already trained base model for MNIST dataset
base_model_path = "/content/drive/MyDrive/ANN/model.h5"
base_model = tf.keras.models.load_model(base_model_path)
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (Flatten)        (None, 784)               0         
                                                                 
 hiddenLayer1 (Dense)        (None, 300)               235500    
                                                                 
 hiddenLayer2 (Dense)        (None, 100)               30100     
                                                                 
 outputLayer (Dense)         (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Now base model has 784 input neurons after flattening 
  # then we have 300 unit of neurons 
  # then we have 100 neurons 
  # at the final layer we are having 10 neurons 
# In transfer learning we have this model already trained. We will simply take this base and freeze the weights 
# These weights will become non trainable. We will replace the last layer with two neurons as we have to classify 
# now the numbers only as odd or even unlike in the previous case where we needed to classify it among the ten digits

# Freeze weights 
for layer in base_model.layers[: -1]:
        print(f"before freezing weights {layer.name}: {layer.trainable}") 
        layer.trainable = False
        print(f"after freezing weights {layer.name}: {layer.trainable}") 


before freezing weights inputLayer: True
after freezing weights inputLayer: False
before freezing weights hiddenLayer1: True
after freezing weights hiddenLayer1: False
before freezing weights hiddenLayer2: True
after freezing weights hiddenLayer2: False


In [6]:
# Modify the last layer for our use case of segregating as even or odd 

base_layers = base_model.layers[:-1]

new_model = tf.keras.models.Sequential(base_layers)
new_model.add(
    tf.keras.layers.Dense(2, activation="softmax", name="output_layer")
)

new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (Flatten)        (None, 784)               0         
                                                                 
 hiddenLayer1 (Dense)        (None, 300)               235500    
                                                                 
 hiddenLayer2 (Dense)        (None, 100)               30100     
                                                                 
 output_layer (Dense)        (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-trainable params: 265,600
_________________________________________________________________


In [7]:
# We need to take in the training data for the last layer now but the y we have has a ten fold classification 
# We need to update this data as odd and even 

def update_odd_even_labels(labels):
    # odd = 0
    # even = 1
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label%2 == 0, 1, 0)
    return labels

# get the data
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

y_train_bin, y_test_bin, y_valid_bin = update_odd_even_labels([y_train, y_test, y_valid])

11501568/11490434 [==============================] - 0s 0us/step


In [8]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 0.2964 - accuracy: 0.8827 - val_loss: 0.2149 - val_accuracy: 0.9242 - 5s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.2009 - accuracy: 0.9290 - val_loss: 0.1778 - val_accuracy: 0.9392 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 3s - loss: 0.1731 - accuracy: 0.9401 - val_loss: 0.1587 - val_accuracy: 0.9464 - 3s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.1571 - accuracy: 0.9466 - val_loss: 0.1474 - val_accuracy: 0.9500 - 4s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.1463 - accuracy: 0.9508 - val_loss: 0.1397 - val_accuracy: 0.9538 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.1385 - accuracy: 0.9535 - val_loss: 0.1331 - val_accuracy: 0.9556 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.1325 - accuracy: 0.9558 - val_loss: 0.1284 - val_accuracy: 0.9574 - 4s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 3s - loss: 0.1278 - accuracy: 0.9574 - val_loss: 0.1249 - val_accuracy: 0.9586 - 3s/epoch - 2ms/step
Epoch 9/

In [10]:
new_model.evaluate(X_test, y_test_bin)

new_model_path = os.path.join("/content/drive/MyDrive/ANN/", "new_model.h5")
new_model.save(new_model_path)

313/313 [==============================] - 1s 2ms/step - loss: 0.1280 - accuracy: 0.9549
